<a href="https://colab.research.google.com/github/Zontafor/quantum-software/blob/main/ultraviolet_final_project_debugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit
!pip install numpy
!pip install qiskit-aer


import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Operator
from typing import List

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 66.8 MB/s eta 0:00:00


In [6]:
# Prepare initial state to vector b
def prepare_initial_state(circuit, qubits, vector):
    norm = np.linalg.norm(vector)
    theta = 2 * np.arccos(vector[0] / norm)
    circuit.ry(theta, qubits[0])
    if len(vector) > 1 and vector[1] != 0:
        phi = np.angle(vector[1])
        circuit.rz(phi, qubits[0])

def PhaseEstimate(b: QuantumRegister, clock: QuantumRegister, unitary: List[List[complex]]) -> QuantumCircuit:
    circuit = QuantumCircuit(b, clock)
    circuit.h(clock)

    # Constructs the controlled unitary gate
    unitaryGate = Operator(unitary).to_instruction()
    controlled_unitaryGate = unitaryGate.control()

    for i in range(clock.size):
        for _ in range(int(2 ** i)):
            circuit.append(controlled_unitaryGate, [clock[i]] + list(b))

    qft = QFT(inverse=True, num_qubits=clock.size).to_gate()
    circuit.append(qft, clock)
    return circuit

def InversePhaseEstimate(b: QuantumRegister, clock: QuantumRegister, unitary: List[List[complex]]) -> QuantumCircuit:
    circuit = QuantumCircuit(b, clock)

    qft = QFT(inverse=False, num_qubits=clock.size).to_gate()
    circuit.append(qft, clock)

    inverseCheckedUnitary = np.linalg.inv(unitary)
    unitaryGate = Operator(inverseCheckedUnitary).to_instruction()
    controlled_unitaryGate = unitaryGate.control()

    for i in range(clock.size):
        for _ in range(int(2 ** i)):
            circuit.append(controlled_unitaryGate, [clock[i]] + list(b))

    circuit.h(clock)
    return circuit

def controlled_rotation(qc: QuantumCircuit, clock_qubits: QuantumRegister, ancilla_qubit: QuantumRegister) -> QuantumCircuit:
    for i in range(clock_qubits.size):
        angle = 2 * np.arcsin(i / (clock_qubits.size - 1))
        qc.cry(angle, clock_qubits[i], ancilla_qubit[0])
    return qc

def main():
    A = (1 / np.sqrt(2)) * np.array([[1, -1], [1, 1]])
    b = np.array([1, 0])

    norm_b = np.linalg.norm(b)
    b_normalized = b / norm_b

    n = A.shape[0]
    num_qubits = int(np.ceil(np.log2(n)))

    b_qubits = QuantumRegister(num_qubits, name='b')
    clock_qubits = QuantumRegister(n, name='clock')
    ancilla_qubit = QuantumRegister(1, name='ancilla')
    classical_reg = ClassicalRegister(n, name='measure')

    qc = QuantumCircuit(b_qubits, clock_qubits, ancilla_qubit, classical_reg)

    prepare_initial_state(qc, b_qubits, b_normalized)

    phase_estimation_circuit = PhaseEstimate(b_qubits, clock_qubits, A)
    qc.compose(phase_estimation_circuit, inplace=True)

    controlled_rotation(qc, clock_qubits, ancilla_qubit)

    inverse_phase_estimation_circuit = InversePhaseEstimate(b_qubits, clock_qubits, A)
    qc.compose(inverse_phase_estimation_circuit, inplace=True)

    qc.measure(b_qubits, classical_reg)

    simulator = AerSimulator()
    transpiled = transpile(qc, simulator)
    result = simulator.run(transpiled, shots=1024).result()
    counts = result.get_counts()

    print("Measurement Results:", counts)

if __name__ == "__main__":
    main()

Measurement Results: {'11': 249, '00': 775}
